<p style="text-align:center;">
    <font face="Georgia" size=5 color="green"><b>importing libraries</b></font>
</p>

In [1]:
import tensorflow as tf
import numpy as np

<p style="text-align:center;">
    <font face="Georgia" size=5 color="green"><b>defining Contrastive Loss Function</b></font>
</p>

In [2]:
def contrastive_loss(y_true, y_pred, margin=1.0, similar_weight=1.0, dissimilar_weight=0.5):
    """
    Calculates the contrastive loss.

    """
    y_true = tf.cast(y_true, tf.float32)
    y_true = tf.reshape(y_true, [-1, 1])

    # Calculate the Euclidean distance
    distance = tf.sqrt(tf.reduce_sum(tf.square(y_pred[0] - y_pred[1]), axis=-1, keepdims=True) + 1e-7)
    distance = tf.clip_by_value(distance, 1e-7, 1e7)

    # Compute the contrastive loss
    loss_similar = y_true * similar_weight * tf.square(distance)
    loss_dissimilar = (1.0 - y_true) * dissimilar_weight * tf.square(tf.maximum(margin - distance, 0.0))

    # Combine and average over the batch
    loss = tf.reduce_mean(loss_similar + loss_dissimilar)

    return loss

<p style="text-align:center;">
    <font face="Georgia" size=5 color="green"><b>defining Contrastive Loss Function</b></font>
</p>

In [3]:
# --- Input Data from the Image ---
matrix1_list = [
    [0, 1, 4],
    [7, 2, 1],
    [2, 1, 4],
    [1, 2, 0]
]

matrix2_list = [
    [0, 2, 1],
    [0, 2, 1],
    [3, 0, 1],
    [1, 2, 0]
]

# Convert lists to TensorFlow tensors with float32 type
tensor1 = tf.constant(matrix1_list, dtype=tf.float32)
tensor2 = tf.constant(matrix2_list, dtype=tf.float32)

# Flatten the tensors into vectors (embeddings)
vector1 = tf.reshape(tensor1, [-1])
vector2 = tf.reshape(tensor2, [-1])

# Prepare the y_pred input for the loss function
y_pred_input = [vector1, vector2]

In [4]:
# --- Calculate Loss ---
# Scenario 1: Assume the pair is SIMILAR (y_true = 1)
y_true_similar = tf.constant([1.0], dtype=tf.float32)
loss_val_similar = contrastive_loss(y_true_similar, y_pred_input)

# Scenario 2: Assume the pair is DISSIMILAR (y_true = 0)
y_true_dissimilar = tf.constant([0.0], dtype=tf.float32)
loss_val_dissimilar = contrastive_loss(y_true_dissimilar, y_pred_input)

In [5]:
# --- Print Results ---
print("--- Contrastive Loss Calculation using Code 1 ---")
print(f"Input Vector 1 (flattened): {vector1.numpy()}")
print(f"Input Vector 2 (flattened): {vector2.numpy()}")
print("-" * 20)
# Calculate distance separately for clarity
distance_val = tf.sqrt(tf.reduce_sum(tf.square(vector1 - vector2)) + 1e-7)
print(f"Euclidean Distance between vectors: {distance_val.numpy():.4f}")
print(f"Margin: {1.0}")
print(f"Similar Pair Weight: {1.0}")
print(f"Dissimilar Pair Weight: {0.5}")
print("-" * 20)
print(f"\033[92mLoss assuming the pair is SIMILAR (y_true=1): {loss_val_similar.numpy():.4f}\033[0m")
print(f"\033[92mLoss assuming the pair is DISSIMILAR (y_true=0): {loss_val_dissimilar.numpy():.4f}\033[0m")

--- Contrastive Loss Calculation using Code 1 ---
Input Vector 1 (flattened): [0. 1. 4. 7. 2. 1. 2. 1. 4. 1. 2. 0.]
Input Vector 2 (flattened): [0. 2. 1. 0. 2. 1. 3. 0. 1. 1. 2. 0.]
--------------------
Euclidean Distance between vectors: 8.3666
Margin: 1.0
Similar Pair Weight: 1.0
Dissimilar Pair Weight: 0.5
--------------------
Loss assuming the pair is SIMILAR (y_true=1): 70.0000
Loss assuming the pair is DISSIMILAR (y_true=0): 0.0000
